# Inverse Kinematics and PID control
In this homework assignemnt, you will be tasked with creating an IK solver and controlling a robot arm using PID control (specifically the Franka Panda 7DoF robot). We will be creating two versions of the controller, one that uses pure velocity control and another that using torque control with feedforward dynamics. 

### Import mujoco and numpy

In [1]:
import mujoco
import mujoco_viewer
import numpy as np
from numpy.linalg import inv, pinv
import scipy as sp

from utilts import rot_err, getSitePose, getSiteVel, getSiteJacobian
from autograder import autograder
from sim import Sim

franka_sim = Sim(fname='./franka_emika_panda/scene_torque_actuators.xml')

In [2]:

def getMassMatrix(model, data):
    """
        This function takes in a mujoco model and data class and returns
        the full system mass/inertia matrix 
        M(q) : nxn matrix where n = dim(q) <- note that for floating base system, this includes 6 dim base degree DoF
    """
    mass_matrix = np.zeros((model.nv, model.nv))
    mujoco.mj_fullM(model, mass_matrix, data.qM)
    return mass_matrix

def getBiasTerms(model, data):
    """
        This function takes in a mujoco model and data class and returns
        the dynamic bias terms e.g., friction, gravity, coriolis, etc. 
        b(q,qdot) : nx1 array
    """
    return data.qfrc_bias

def getOSMassMatrix(sys_mass_matrix, jac):
    """
        This function takes in the mass matrix and a jacobian matrix and returns 
        the operational space matrix M_x(q) = (J M^{-1} J.T)^{-1}
    """
    return inv(jac@np.linalg.inv(sys_mass_matrix)@jac.T)




In [3]:
def operational_space_control(model, data):
    # PD control parameters
    kp = 350.
    kd = 50.
    # simulate and render
    (R_ee, p_ee)     = getSitePose(model, data, "ee_site")
    (R_des, p_des)   = getSitePose(model, data, "target_site")
    jac              = getSiteJacobian(model, data, "ee_site")

    ## we can now work directly with errors in the operational space
    err_p   = p_des - p_ee
    err_rot = rot_err(R_des, R_ee)
    err     = np.concatenate([err_p, err_rot])

    ### to compute velocity in operational space, we do the following transform 
    ### this is equivalent to J qdot
    err_dot = jac@data.qvel

    ### calculate the xddot_des in operational space 
    xddot_des = kp * err + kd *(-err_dot)

    ### pull out the dynamic terms 
    mass_matrix = getMassMatrix(model, data)
    bias        = getBiasTerms(model, data)

    ### calculate the operational space mass matrix 
    op_mass_matrix = getOSMassMatrix(mass_matrix, jac)

    ### compute the torque controller with respect to the operational space 
    ### that is J.T (M_x xddot_des) + b
    tau = jac.T @ op_mass_matrix@ xddot_des + bias
    return tau

In [4]:
def simulateCtrl(sim, ctrl_method, render=True, t_max=1000):
    if len(sim.model.key_qpos) != 0:
        sim.data.qpos = sim.model.key_qpos
    mujoco.mj_step(sim.model, sim.data)
    log = []
    if render:
        viewer = mujoco_viewer.MujocoViewer(sim.model, sim.data)
        viewer.vopt.frame = 3
    for t in range(t_max):
        u = ctrl_method(sim.model, sim.data)
        sim.data.ctrl = u 
        mujoco.mj_step(sim.model, sim.data)
        if render:
            if viewer.is_alive:
                viewer.render()
    if render:
        viewer.close()

In [5]:
simulateCtrl(franka_sim, operational_space_control)

Pressed ESC
Quitting.
